In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from astropy.units import one
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Flatten , Dropout, Activation, BatchNormalization, Input
from tensorflow.keras.layers import LSTM, GRU, CuDNNLSTM, CuDNNGRU
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import concatenate as Concatenate
from tensorflow.keras.models import Model
from tensorflow.contrib.opt import MomentumWOptimizer, AdamWOptimizer
from tensorflow.keras.regularizers import l2
#from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.keras.models import Sequential
from sklearn.metrics import confusion_matrix
from tensorflow.keras.optimizers import Adam
import itertools
import random
import math
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score, precision_recall_curve, average_precision_score, roc_auc_score, roc_curve, auc

from IPython.display import clear_output

from tensorflow.keras import backend as K
#from tensorflow.keras.engine.topology import Layer

from tensorflow.keras.layers import Layer 
from tensorflow.keras import initializers as initializers, regularizers, constraints
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, SpatialDropout1D, Bidirectional, concatenate, InputSpec

In [21]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [22]:
# # memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize

In [23]:

is_on_colab = False

is_show_colab_env_info = True



if(is_on_colab):

    if(is_show_colab_env_info):
        import psutil
        import humanize
        import os
        import GPUtil as GPU
        GPUs = GPU.getGPUs()
        # XXX: only one GPU on Colab and isn’t guaranteed
        gpu = GPUs[0]
        
    def printm():
        process = psutil.Process(os.getpid())
        print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
        print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

        #GPU count and name
        !lscpu |grep 'Model name'
        !nvidia-smi -L
        printm()
  
  
    from google.colab import drive
    drive.mount('/content/drive')

    gdrive_home_dir = '/content/drive/My Drive/'

    project_dir = f'{gdrive_home_dir}DeepCNC/'
    ds_dir = f'{project_dir}data/'
    snapshot_dir = f'{project_dir}snapshots/'

    !ls '{ds_dir}'

else:
    project_dir = f'/disks/data/paper_projects/archive/DeepCNC/'
    ds_dir = f'{project_dir}data/'
    snapshot_dir = f'{project_dir}snapshots/'


  

In [24]:
# project_dir = '../'
# ds_dir = f'{project_dir}data/'
# snapshot_dir = f'{project_dir}snapshots/'
    
#fnmInput = f'{ds_dir}inputSeq.txt';
fnmInput = f'{ds_dir}inputSeq_N_ReplacedBy_C.txt';

fnmLabel= f'{ds_dir}inputLabel.txt'

# fnmInput = f'{ds_dir}input.txt'; 
# fnmLabel= f'{ds_dir}inputLabel_0-1.txt'


# input_original.txt has 5 nucleotides, so we should use input.txt
# inputLabel.txt's contents (0 and 1) need to be swapped because of Keras


# fnmInput='./data/input_Test.txt';
# fnmLabel ='./data/inputLabel_Test.txt'

In [25]:
with open(fnmInput,'r') as f:
    print(len(f.readlines()))
    
    
with open(fnmLabel,'r') as f:
    print(len(f.readlines()))

37274
37274


In [26]:
########################### Stratified k-fold Cross validation: IMBALANCED SET ###############

# train_features, test_features, train_labels, test_labels = train_test_split(input_features, input_labels,
#                                                                                 test_size=hyper_test_split,
#                                                                                 stratify=input_labels,
#                                                                                 random_state=42)

useFullDataSet = 1;
ratio_High_Low = 10;

with open(fnmLabel) as f:
    labels = [ int(x) for x in f.read().split()]
    
pos_samplecount = labels.count(0)
neg_samplecount = labels.count(1)
print(pos_samplecount, neg_samplecount)


totPos = pos_samplecount
totNeg = neg_samplecount 

18787 18487


In [27]:
#totPos = 2796;
#totNeg = 90923;  # 93719- 2796;

totPosNeg = totPos + totNeg;
print("totPos: ", totPos, " totNeg: ", totNeg)
stepSize = 1;

posIndex = [ i for i in range(len(labels)) if labels[i] == 0 ]
negIndex = [ i for i in range(len(labels)) if labels[i] == 1 ]


totPos:  18787  totNeg:  18487


In [28]:
#posIndex = [i for i in range(0, totPos, stepSize)];
#negIndex = [j for j in range(totPos, totPosNeg, stepSize)];

print("Printing index")

sizePos = len(posIndex)
sizeNeg = len(negIndex)

print("size of pos :", sizePos)
print("size of neg :", sizeNeg)

negIndexShuffled = negIndex
random.shuffle(negIndexShuffled)

ratio_High_Low_Max = math.floor(sizeNeg/ sizePos)

if ratio_High_Low > ratio_High_Low_Max:
    ratio_High_Low = ratio_High_Low_Max;

print( ' ration: ' , ratio_High_Low)


if useFullDataSet:
    mergedIndex = posIndex +  negIndexShuffled ;
    print("After selection size of pos :", len(posIndex))
    print("After selection size of neg :", len(negIndexShuffled ))
    print( ' len of merge: ' , len(mergedIndex) )
    
    
else:
    posIndex_Selected = posIndex
    negIndexShuffled_Selected = negIndexShuffled[0:totPos * ratio_High_Low]
    mergedIndex = posIndex_Selected +  negIndexShuffled_Selected ;
    print("After selection size of pos :", len(posIndex_Selected))
    print("After selection size of neg :", len(negIndexShuffled_Selected))
    print( ' len of merged : ' , len(mergedIndex) )

# sequences_New = sequences[mergedIndex]
# labels_New  = labels[mergedIndex] 
# inp = input_features[mergedIndex]

Printing index
size of pos : 18787
size of neg : 18487
 ration:  0
After selection size of pos : 18787
After selection size of neg : 18487
 len of merge:  37274


In [29]:

# sequences_Orig = [ (line.strip())[15:-15] for line in open(fnmInput)] # .rstrip('\n')
#sequences_Orig = [ (line.strip()) for line in open(fnmInput)] # .rstrip('\n')
sequences_Orig = [ line.strip()[:1000] for line in open(fnmInput)] # .rstrip('\n')
sequences_Orig = list(filter(None, sequences_Orig))  # This removes empty sequences.

print('Total Seq found:' , len(sequences_Orig) )

sequences = [ sequences_Orig[i] for i in mergedIndex ]
print('Total Seq found AFTER selection:' , len(sequences) )


# Let's print the first few sequences.
df = pd.DataFrame(sequences_Orig, index=np.arange(1, len(sequences_Orig) + 1), columns=['Sequences'])
# print(df.head())
setNucleotide= set()
for seq in sequences_Orig:
    s = set (list(seq))
    setNucleotide = setNucleotide | s
#   print( len(sequence) )

print("Finally all nucleotides: ")
totalNuclType= setNucleotide.__len__()
print(totalNuclType)

with open(fnmInput ,'rt') as f:
    content = f.readlines()

content = [c.strip() for c in content]
content = "".join(content)
alphabet = list(set(content))
integer_encoder = LabelEncoder()
integer_encoder.fit(alphabet)
print("classes: " ,list(integer_encoder.classes_))


Total Seq found: 37274
Total Seq found AFTER selection: 37274
Finally all nucleotides: 
4
classes:  ['A', 'C', 'G', 'T']


In [30]:
####################  Encode Input Sequence ##############
one_hot_encoder = OneHotEncoder(categories=[range(len(integer_encoder.classes_))])
input_features = []
myBuffer = ''
count =1;
for sequence in sequences:
    integer_encoded = integer_encoder.transform(list(sequence))
    integer_encoded = np.array(integer_encoded).reshape(-1, 1)
    one_hot_encoded = one_hot_encoder.fit_transform(integer_encoded)
    input_features.append(one_hot_encoded.toarray())
    # myBuffer += ("Seq no:" + str(count) + "  len: " + str(len(sequence))  + " array len: " + str( len(one_hot_encoded.toarray()))  + "\n" )
    count = count + 1

print( "Encoding of input DONE")   
    


Encoding of input DONE


In [31]:
print( "Check Encoding Shape")
lengths = [f.shape for f in input_features]
print(set(lengths))    
    
np.set_printoptions(threshold=40)
input_features = np.stack( input_features )

print( len(input_features ) )

print("Example sequence\n-----------------------")
print('DNA Sequence #1:\n' ,sequences[0][:10] ,'...' ,sequences[0][-10:])
print('One hot encoding of Sequence #1:\n' ,input_features[0].T)

Check Encoding Shape
{(1000, 4)}
37274
Example sequence
-----------------------
DNA Sequence #1:
 GAGTGAATTA ... CTGGAAGGCT
One hot encoding of Sequence #1:
 [[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [1. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [32]:
####################  Encode Label ##############

# def processLabel():
# LABELS_URL = 'https://raw.githubusercontent.com/abidlabs/deep-learning-genomics-primer/master/labels.txt'
# labels = requests.get(LABELS_URL).text.split('\n')
# labels = list(filter(None, labels))  # removes empty sequences
labels_Orig = [line.rstrip('\n') for line in open(fnmLabel)]
labels_Orig = list(filter(None, labels_Orig))  # This removes empty sequences.

labels = [ labels_Orig[i] for i in mergedIndex]

one_hot_encoder = OneHotEncoder(categories='auto')
labels = np.array(labels).reshape(-1, 1)
input_labels = one_hot_encoder.fit_transform(labels).toarray()

print('Labels:\n', labels.T)
print('One-hot encoded labels:\n', input_labels.T)
print( len(input_labels ) )

labels = np.array(list(map(int, labels)))  # added for sigmoid activation

####################  Encode Label End ##############

Labels:
 [['0' '0' '0' ... '1' '1' '1']]
One-hot encoded labels:
 [[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 1.]]
37274


In [33]:
k_folds=  10 # 10 # 2
hyper_epoch=  500 #50 # 5
class_weight = {0: 1., 1: 1.} # 10 

#class_weight = {0: ratio_High_Low, 1: 1.} # 10 
#print(ratio_High_Low)

hyper_classes=2

hyper_validation_split = 0.20
#hyper_test_split=0.10
#hyper_pool_size = (1,2)
hyper_batch_size = 256
# hyper_optimizer = 'Adadelta'
# hyper_init_mode = 'zero'
# hyper_activation = 'tanh'
# hyper_filter = 50
# hyper_filterL = 30
# hyper_filterW = 32
# hyper_neurons = 32
# hyper_dropout_rate = 0.1


model_history = []
dfHistory = pd.DataFrame( columns=['Precsion' , 'Recall' , 'Specificity' , 'Accuracy'])


In [34]:
# updatable plot

# https://gist.github.com/stared/dfb4dfaf6d9a8501cd1cc8b8cb806d2e

class PlotLosses(Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        #clear_output(wait=True)
        plt.cla()
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show();


In [35]:
# import keras.backend as K

# # https://keras.io/metrics/
# def my_pred(y_true, y_pred):
#     #return K.mean(y_pred)

# #     cm = confusion_matrix(np.argmax(y_true, axis=1), np.argmax(y_pred, axis=1))
#     cm = confusion_matrix(y_true, y_pred)
#     #print('Confusion matrix:\n' ,cm)
#     # cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
#     total1=sum(sum(cm))
#     accuracy1=(tp+tn)/total1
#     sensitivity1 = tp/(tp+fn)
#     specificity1 = tn/(fp+tn)
#     if (tp)==0:
#         precision1 = 0
#     else:
#         precision1 = tp / (tp + fp)

#     MCC = (tp*tn - fp*fn) / np.sqrt(  (tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)  ) 
#     return MCC

# # model.compile(optimizer='rmsprop',  loss='binary_crossentropy',   metrics=['accuracy', my_pred])


In [36]:
from tensorflow.keras.callbacks import Callback
import tensorflow.keras.backend as K
import numpy as np

class SGDRScheduler(Callback):
    '''Cosine annealing learning rate scheduler with periodic restarts.
    # Usage
        ```python
            schedule = SGDRScheduler(min_lr=1e-5,
                                     max_lr=1e-2,
                                     steps_per_epoch=np.ceil(epoch_size/batch_size),
                                     lr_decay=0.9,
                                     cycle_length=5,
                                     mult_factor=1.5)
            model.fit(X_train, Y_train, epochs=100, callbacks=[schedule])
        ```
    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        lr_decay: Reduce the max_lr after the completion of each cycle.
                  Ex. To reduce the max_lr by 20% after each cycle, set this value to 0.8.
        cycle_length: Initial number of epochs in a cycle.
        mult_factor: Scale epochs_to_restart after each full cycle completion.
    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: http://arxiv.org/abs/1608.03983
    '''
    def __init__(self,
                 min_lr,
                 max_lr,
                 steps_per_epoch,
                 lr_decay=1,
                 cycle_length=10,
                 mult_factor=2):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.lr_decay = lr_decay

        self.batch_since_restart = 0
        self.next_restart = cycle_length

        self.steps_per_epoch = steps_per_epoch

        self.cycle_length = cycle_length
        self.mult_factor = mult_factor

        self.history = {}

    def clr(self):
        '''Calculate the learning rate.'''
        fraction_to_restart = self.batch_since_restart / (self.steps_per_epoch * self.cycle_length)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(fraction_to_restart * np.pi))
        return lr

    def on_train_begin(self, logs={}):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.max_lr)

    def on_batch_end(self, batch, logs={}):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        self.batch_since_restart += 1
        K.set_value(self.model.optimizer.lr, self.clr())
    
    def on_epoch_end(self, epoch, logs={}):
        '''Check for end of current cycle, apply restarts when necessary.'''
        if epoch + 1 == self.next_restart:
            self.batch_since_restart = 0
            self.cycle_length = np.ceil(self.cycle_length * self.mult_factor)
            self.next_restart += self.cycle_length
            self.max_lr *= self.lr_decay
            self.best_weights = self.model.get_weights()

        #clear_output(wait=True)
        plt.cla()
        plt.plot(self.history['lr'], label="learning rate")
        plt.legend()
        plt.show();   
        
        
    def on_train_end(self, logs={}):
        '''Set weights to the values from the end of the most recent cycle for best performance.'''
        self.model.set_weights(self.best_weights)


In [37]:
def make_model():
    
    # -------------------------------------------------------------------------------------------------------------------------------------
    # ------------------------------------------ sequential definition --------------------------------------------------------------------        
    # -------------------------------------------------------------------------------------------------------------------------------------
    def make_seq_cnn():
        # regularization: https://machinelearningmastery.com/how-to-reduce-overfitting-in-deep-learning-with-weight-regularization/

        model = Sequential()

        #model.add(Conv1D(filters=32, kernel_size=8 ,input_shape=( inputSeqLength, totalNuclType ))  )
        model.add(Conv1D(filters=64, kernel_size=12 ,input_shape=( inputSeqLength, totalNuclType ), kernel_regularizer=l2(4e-4), bias_regularizer=l2(4e-4) )  )    
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(rate = 0.5)) # input = 129 * 32
        model.add(MaxPooling1D(pool_size= 2 ))#totalNuclType)) # input = 129 * 32 , output= 32*32 (filter/pool_size* filterTotal)

        #model.add(Conv1D(filters=32, kernel_size=8))#, input_shape=(32, 32))) # inputSeqLength, totalNuclType
        model.add(Conv1D(filters=48, kernel_size=12, kernel_regularizer=l2(4e-4), bias_regularizer=l2(4e-4) )  )    
        model.add(BatchNormalization())    
        model.add(Activation('relu'))
        model.add(Dropout(rate = 0.5)) # (32-8)/1  + 1 = 25 , output = 25 *32
        model.add(MaxPooling1D(pool_size=2 ))#totalNuclType))  # input = 25 *32 , output 6 * 32

#         #model.add(Conv1D(filters=32, kernel_size=8))#, input_shape=(32, 32))) # inputSeqLength, totalNuclType
#         model.add(Conv1D(filters=32, kernel_size=12, kernel_regularizer=l2(4e-4), bias_regularizer=l2(4e-4) )  )    
#         model.add(BatchNormalization())    
#         model.add(Activation('relu'))
#         model.add(Dropout(rate = 0.4)) # (32-8)/1  + 1 = 25 , output = 25 *32
#         model.add(MaxPooling1D(pool_size = 2 ))#totalNuclType))  # input = 25 *32 , output 6 * 32    

    #     model.add(Conv1D(filters=92, kernel_size=3))#, input_shape=(32, 32))) # inputSeqLength, totalNuclType
    #     #model.add(Conv1D(filters=32, kernel_size=12 ,input_shape=( inputSeqLength, totalNuclType ), kernel_regularizer=l2(4e-5), bias_regularizer=l2(4e-5) )  )    
    #     #model.add(BatchNormalization())    
    #     model.add(Activation('relu'))
    #     model.add(Dropout(rate = 0.4)) # (32-8)/1  + 1 = 25 , output = 25 *32
    #     model.add(MaxPooling1D(pool_size=totalNuclType))  # input = 25 *32 , output 6 * 32  


        #model.add(Bidirectional(LSTM(16, return_sequences=True)))
        #model.add(Bidirectional(LSTM(16, return_sequences=True, kernel_regularizer=l2(1e-6), recurrent_regularizer=l2(1e-6), bias_regularizer=l2(1e-6)) ))
        #model.add(Dropout(rate = 0.4))    

        model.add(Flatten())

        #model.add(Dense(16))
        #  cite: use weight decay increasing from 0 to 1E-5 at 1200 epochs, to 1E-4 at 2500 epochs, and to 1E-3 at 400 epochs. 
        # […] The surface is smoother and transitions are more gradual
        model.add(Dense(32, kernel_regularizer=l2(0.02) ) )
        model.add(BatchNormalization())    
        model.add(Activation('relu'))
        model.add(Dropout(rate = 0.6)) 
        model.add(Dense(2, kernel_regularizer=l2(0.02)))        
        model.add(Activation('softmax'))
        
        SGDRScheduler_config = { 'min_lr':2e-4,
                                 'max_lr':.2e-3,
                                 'steps_per_epoch':np.ceil(hyper_epoch/(hyper_batch_size)),
                                 'lr_decay':0.95,
                                 'cycle_length': 5,
                                 'mult_factor': 1.5}
        
        config = {'learning_rate':0.001, 'SGDRScheduler_config': SGDRScheduler_config}
        
        return [model, config]

    
    # -------------------------------------------------------------------------------------------------------------------------------------
    # ------------------------------------------ sequential definition --------------------------------------------------------------------        
    # -------------------------------------------------------------------------------------------------------------------------------------
    def make_seq_3mer_cnn():
        # regularization: https://machinelearningmastery.com/how-to-reduce-overfitting-in-deep-learning-with-weight-regularization/

        model = Sequential()

        model.add(Conv1D(filters=92, kernel_size=3 ,input_shape=( inputSeqLength, totalNuclType ), kernel_regularizer=l2(4e-4), bias_regularizer=l2(4e-4) )  )    
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(rate = 0.5)) # input = 129 * 32
        model.add(MaxPooling1D(pool_size= 2 ))#totalNuclType)) # input = 129 * 32 , output= 32*32 (filter/pool_size* filterTotal)

        #model.add(Conv1D(filters=32, kernel_size=8))#, input_shape=(32, 32))) # inputSeqLength, totalNuclType
        model.add(Conv1D(filters=48, kernel_size=3, kernel_regularizer=l2(4e-4), bias_regularizer=l2(4e-4) )  )    
        model.add(BatchNormalization())    
        model.add(Activation('relu'))
        model.add(Dropout(rate = 0.5)) # (32-8)/1  + 1 = 25 , output = 25 *32
        model.add(MaxPooling1D(pool_size=2 ))#totalNuclType))  # input = 25 *32 , output 6 * 32

#         #model.add(Conv1D(filters=32, kernel_size=8))#, input_shape=(32, 32))) # inputSeqLength, totalNuclType
#         model.add(Conv1D(filters=32, kernel_size=12, kernel_regularizer=l2(4e-4), bias_regularizer=l2(4e-4) )  )    
#         model.add(BatchNormalization())    
#         model.add(Activation('relu'))
#         model.add(Dropout(rate = 0.4)) # (32-8)/1  + 1 = 25 , output = 25 *32
#         model.add(MaxPooling1D(pool_size = 2 ))#totalNuclType))  # input = 25 *32 , output 6 * 32    

    #     model.add(Conv1D(filters=92, kernel_size=3))#, input_shape=(32, 32))) # inputSeqLength, totalNuclType
    #     #model.add(Conv1D(filters=32, kernel_size=12 ,input_shape=( inputSeqLength, totalNuclType ), kernel_regularizer=l2(4e-5), bias_regularizer=l2(4e-5) )  )    
    #     #model.add(BatchNormalization())    
    #     model.add(Activation('relu'))
    #     model.add(Dropout(rate = 0.4)) # (32-8)/1  + 1 = 25 , output = 25 *32
    #     model.add(MaxPooling1D(pool_size=totalNuclType))  # input = 25 *32 , output 6 * 32  


        #model.add(Bidirectional(LSTM(16, return_sequences=True)))
        #model.add(Bidirectional(LSTM(16, return_sequences=True, kernel_regularizer=l2(1e-6), recurrent_regularizer=l2(1e-6), bias_regularizer=l2(1e-6)) ))
        #model.add(Dropout(rate = 0.4))    

        model.add(Flatten())

        #model.add(Dense(16))
        #  cite: use weight decay increasing from 0 to 1E-5 at 1200 epochs, to 1E-4 at 2500 epochs, and to 1E-3 at 400 epochs. 
        # […] The surface is smoother and transitions are more gradual
        model.add(Dense(48, kernel_regularizer=l2(0.02) ) )
        model.add(BatchNormalization())    
        model.add(Activation('relu'))
        model.add(Dropout(rate = 0.6)) 
        model.add(Dense(2, kernel_regularizer=l2(0.02)))        
        model.add(Activation('softmax'))
        
        SGDRScheduler_config = { 'min_lr':2e-4,
                                 'max_lr':5e-4,
                                 'steps_per_epoch':np.ceil(hyper_epoch/(hyper_batch_size)),
                                 'lr_decay':0.95,
                                 'cycle_length': 5,
                                 'mult_factor': 1.5}
        
        config = {'learning_rate':0.001, 'SGDRScheduler_config': SGDRScheduler_config}
        
        return [model, config]    
    
    # -------------------------------------------------------------------------------------------------------------------------------------
    # ------------------------------------------ sequential definition --------------------------------------------------------------------
    # -------------------------------------------------------------------------------------------------------------------------------------
    def make_seq_cnn_blstm():
        
        # regularization: https://machinelearningmastery.com/how-to-reduce-overfitting-in-deep-learning-with-weight-regularization/

        model = Sequential()

        #model.add(Conv1D(filters=32, kernel_size=8 ,input_shape=( inputSeqLength, totalNuclType ))  )
        model.add(Conv1D(filters=32, kernel_size=12 ,input_shape=( inputSeqLength, totalNuclType ), kernel_regularizer=l2(4e-4), bias_regularizer=l2(4e-4) )  )    
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(rate = 0.4)) # input = 129 * 32
        model.add(MaxPooling1D(pool_size= 2 ))#totalNuclType)) # input = 129 * 32 , output= 32*32 (filter/pool_size* filterTotal)

        #model.add(Conv1D(filters=32, kernel_size=8))#, input_shape=(32, 32))) # inputSeqLength, totalNuclType
        model.add(Conv1D(filters=32, kernel_size=12, kernel_regularizer=l2(4e-4), bias_regularizer=l2(4e-4) )  )    
        model.add(BatchNormalization())    
        model.add(Activation('relu'))
        model.add(Dropout(rate = 0.4)) # (32-8)/1  + 1 = 25 , output = 25 *32
        model.add(MaxPooling1D(pool_size=2 ))#totalNuclType))  # input = 25 *32 , output 6 * 32

        #model.add(Conv1D(filters=32, kernel_size=8))#, input_shape=(32, 32))) # inputSeqLength, totalNuclType
        model.add(Conv1D(filters=32, kernel_size=12, kernel_regularizer=l2(8e-4), bias_regularizer=l2(8e-4) )  )    
        model.add(BatchNormalization())    
        model.add(Activation('relu'))
        model.add(Dropout(rate = 0.4)) # (32-8)/1  + 1 = 25 , output = 25 *32
        model.add(MaxPooling1D(pool_size =2 ))#totalNuclType))  # input = 25 *32 , output 6 * 32    

    #     model.add(Conv1D(filters=92, kernel_size=3))#, input_shape=(32, 32))) # inputSeqLength, totalNuclType
    #     #model.add(Conv1D(filters=32, kernel_size=12 ,input_shape=( inputSeqLength, totalNuclType ), kernel_regularizer=l2(4e-5), bias_regularizer=l2(4e-5) )  )    
    #     #model.add(BatchNormalization())    
    #     model.add(Activation('relu'))
    #     model.add(Dropout(rate = 0.4)) # (32-8)/1  + 1 = 25 , output = 25 *32
    #     model.add(MaxPooling1D(pool_size=totalNuclType))  # input = 25 *32 , output 6 * 32  


        #model.add(Bidirectional(LSTM(16, return_sequences=True)))
        model.add(Bidirectional(CuDNNGRU(240, return_sequences=True, kernel_regularizer=l2(4e-3), recurrent_regularizer=l2(5e-3), bias_regularizer=l2(4e-3)) ))
        model.add(Dropout(rate = 0.6))    

        model.add(Flatten())

        #model.add(Dense(16))
        #  cite: use weight decay increasing from 0 to 1E-5 at 1200 epochs, to 1E-4 at 2500 epochs, and to 1E-3 at 400 epochs. 
        # […] The surface is smoother and transitions are more gradual
        model.add(Dense(16, kernel_regularizer=l2(0.01) ) )
        model.add(BatchNormalization())    
        model.add(Activation('relu'))
        model.add(Dropout(rate = 0.5)) 
        model.add(Dense(2, kernel_regularizer=l2(0.01)))        
        model.add(Activation('softmax'))
        
#         SGDRScheduler_config = { 'min_lr':2e-4,
#                                  'max_lr':.2e-3,
#                                  'steps_per_epoch':np.ceil(hyper_epoch/(hyper_batch_size)),
#                                  'lr_decay':0.95,
#                                  'cycle_length': 5,
#                                  'mult_factor': 1.5}

#         SGDRScheduler_config = { 'min_lr':5e-4,
#                                  'max_lr':.5e-3,
#                                  'steps_per_epoch':np.ceil(hyper_epoch/(hyper_batch_size)),
#                                  'lr_decay':0.95,
#                                  'cycle_length': 2,
#                                  'mult_factor': 1.5}        


        SGDRScheduler_config = { 'min_lr':1e-4,
                                 'max_lr': .5e-3,
                                 'steps_per_epoch':np.ceil(hyper_epoch/(hyper_batch_size)),
                                 'lr_decay':0.95,
                                 'cycle_length': 2,
                                 'mult_factor': 1.5}        
        
        config = {'learning_rate':0.002, 'SGDRScheduler_config': SGDRScheduler_config}
        
        return [model, config]    
    
    # -------------------------------------------------------------------------------------------------------------------------------------
    # ------------------------------------------ functional definition --------------------------------------------------------------------        
    # -------------------------------------------------------------------------------------------------------------------------------------
    
    def make_func_inception():
        input_2d = Input(shape = ( inputSeqLength, totalNuclType ))

        def create_inception_module(input_tensor):
            c3 = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(input_tensor)
            c5 = Conv1D(filters=64, kernel_size=5, padding='same', activation='relu')(input_tensor)
            c8 = Conv1D(filters=32, kernel_size=8, padding='same', activation='relu')(input_tensor)
            #c12 = Conv1D(filters=32, kernel_size=12, padding='same', activation='relu')(input_tensor)
            #inception_module = Concatenate([c3, c5, c8, c12], axis = 2)
            inception_module = Concatenate([c3, c5, c8], axis = 2)
            
            
            #p3 = MaxPooling1D(pool_size=2, strides=1, padding='same')(input_tensor)
            #pool = MaxPooling1D(pool_size = totalNuclType, strides=1, padding='same')(c8)
            #inception_module = c8

            return inception_module

        out = create_inception_module(input_2d)
        #out = Dropout(rate = 0.1)(out)  # was 0.4
        out = Dropout(rate = 0.4)(out)  # was 0.4
        out = MaxPooling1D(pool_size = totalNuclType)(out)


        out = create_inception_module(out)
        #out = Dropout(rate = 0.1)(out) # was 0.4
        out = Dropout(rate = 0.4)(out) # was 0.4
        out = MaxPooling1D(pool_size = totalNuclType)(out)

        out = Conv1D(filters=32, kernel_size=8)(out)
        out = Activation('relu')(out)     
        #out = Dropout(rate = 0.45)(out) # was 0.4
        out = Dropout(rate = 0.4)(out) # was 0.4
        out = MaxPooling1D(pool_size = totalNuclType)(out)

    #     out = Bidirectional( LSTM(16, return_sequences=True))(out)
    #     out = Bidirectional( LSTM(16, return_sequences=True))(out)    
    #     out = Dropout(rate = 0.25)(out)

        out = Flatten()(out)
        out = Dense(16)(out)
        out = Activation('relu')(out)
        out = Dropout(rate = 0.5)(out)

        #out = Dense(2, activation='softmax')(out)
        out = Dense(1, activation='sigmoid')(out)

        model = Model(inputs = input_2d, outputs = out)        
    
        SGDRScheduler_config = { 'min_lr':3e-5,
                                 'max_lr':1e-3,    #0.5e-3, #1e-3,
                                 'steps_per_epoch':np.ceil(hyper_epoch/(5*hyper_batch_size)),
                                 'lr_decay':0.95,
                                 'cycle_length':10,
                                 'mult_factor': 1.0}
        
        config = {'learning_rate':0.002, 'SGDRScheduler_config': SGDRScheduler_config}
        
        return [model, config]
    
    
    
    # -------------------------------------------------------------------------------------------------------------------------------------
    # ------------------------------------------ functional definition --------------------------------------------------------------------        
    # -------------------------------------------------------------------------------------------------------------------------------------
    
    def make_func_inception_small():
        input_2d = Input(shape = ( inputSeqLength, totalNuclType ))

        def create_inception_module(input_tensor):
            c3 = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(input_tensor)
            c5 = Conv1D(filters=64, kernel_size=5, padding='same', activation='relu')(input_tensor)
            c8 = Conv1D(filters=32, kernel_size=8, padding='same', activation='relu')(input_tensor)
            #c12 = Conv1D(filters=32, kernel_size=12, padding='same', activation='relu')(input_tensor)
            #inception_module = Concatenate([c3, c5, c8, c12], axis = 2)
            inception_module = Concatenate([c3, c5, c8], axis = 2)
            
            
            #p3 = MaxPooling1D(pool_size=2, strides=1, padding='same')(input_tensor)
            #pool = MaxPooling1D(pool_size = totalNuclType, strides=1, padding='same')(c8)
            #inception_module = c8

            return inception_module

        def create_resnet_module(input_tensor):
            c3 = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(input_tensor)
            c5 = Conv1D(filters=64, kernel_size=5, padding='same', activation='relu')(input_tensor)
            c8 = Conv1D(filters=32, kernel_size=8, padding='same', activation='relu')(input_tensor)
            #c12 = Conv1D(filters=32, kernel_size=12, padding='same', activation='relu')(input_tensor)
            #inception_module = Concatenate([c3, c5, c8, c12], axis = 2)
            inception_module = Concatenate([c3, c5, c8, input_tensor], axis = 2)
            
            
            #p3 = MaxPooling1D(pool_size=2, strides=1, padding='same')(input_tensor)
            #pool = MaxPooling1D(pool_size = totalNuclType, strides=1, padding='same')(c8)
            #inception_module = c8

            return inception_module        
        
        out = create_inception_module(input_2d)
        #out = Dropout(rate = 0.1)(out)  # was 0.4
        out = Dropout(rate = 0.4)(out)  # was 0.4
        out = MaxPooling1D(pool_size = totalNuclType)(out)
        
        out = create_inception_module(out)
        #out = Dropout(rate = 0.1)(out) # was 0.4
        out = Dropout(rate = 0.5)(out) # was 0.4
        out = MaxPooling1D(pool_size = totalNuclType)(out)   

#         out = create_resnet_module(out)
#         #out = Dropout(rate = 0.1)(out) # was 0.4
#         out = Dropout(rate = 0.4)(out) # was 0.4
#         out = MaxPooling1D(pool_size = totalNuclType)(out)   
        
        out = Conv1D(filters=64, kernel_size=8, kernel_regularizer=l2(8e-4), bias_regularizer=l2(8e-4))(out)
        #out = BatchNormalization()(out)
        out = Activation('relu')(out)     
        #out = Dropout(rate = 0.45)(out) # was 0.4
        out = Dropout(rate = 0.6)(out) # was 0.5, and 0.4 before
        out = MaxPooling1D(pool_size = totalNuclType)(out)

        
#         out = Bidirectional( LSTM(16, return_sequences=True))(out)
#         out = Bidirectional( LSTM(16, return_sequences=True))(out)    
#         out = Dropout(rate = 0.25)(out)

        out = Flatten()(out)
        out = Dense(16, kernel_regularizer=l2(0.02))(out)
        out = BatchNormalization()(out)
        out = Activation('relu')(out)
        out = Dropout(rate = 0.7)(out) # was 0.5

        #out = Dense(2, activation='softmax')(out)
        out = Dense(1, activation='sigmoid')(out)

        model = Model(inputs = input_2d, outputs = out)        
    
        SGDRScheduler_config = { 'min_lr':3e-5,
                                 'max_lr':1e-3,    #0.5e-3, #1e-3,
                                 'steps_per_epoch':np.ceil(hyper_epoch/(5*hyper_batch_size)),
                                 'lr_decay':0.85,
                                 'cycle_length':10,
                                 'mult_factor': 1.0}
        
        config = {'learning_rate':0.002, 'SGDRScheduler_config': SGDRScheduler_config}
        
        return [model, config]    
    
    
    return make_func_inception_small()

In [38]:
# from sklearn.metrics import confusion_matrix
# y_true = [2, 0, 2, 2, 0, 1]
# y_pred = [0, 0, 2, 2, 0, 2]
# confusion_matrix(y_true, y_pred)

# y_true = ["cat", "ant", "cat", "cat", "ant", "bird"]
# y_pred = ["ant", "ant", "cat", "cat", "ant", "cat"]
# confusion_matrix(y_true, y_pred, labels=["ant", "bird", "cat"])

# tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0])
# (tn, fp, fn, tp)


In [39]:
########################### Stratified k-fold Cross validation ###############

# train_features, test_features, train_labels, test_labels = train_test_split(input_features, input_labels,
#                                                                                 test_size=hyper_test_split,
#                                                                                 stratify=input_labels,
#                                                                                 random_state=42)

def sensitivity(y_true, y_pred):

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())



class Metrics(Callback):
    
    def __init__(self, val_data):
        super().__init__()
        self.validation_data = val_data
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
        self.val_sensitivity = []
        self.val_specificity = []
    
    def on_train_begin(self, logs={}):
        pass    
    def on_epoch_end(self, epoch, logs={}):
        #print(dir(self.model))
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_targ = self.validation_data[1]
        
        cm = confusion_matrix(val_targ, val_predict)
#         output = ""
#         output = output + 'Confusion matrix:\n' + str(cm) 
#         output += "\n"

#         print('Confusion matrix:\n' ,cm)
        cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
#         total1=sum(sum(cm))
        tn, fp, fn, tp = cm.ravel()
        #tp=cm[0,0];fn=cm[1,0];fp=cm[0,1];tn=cm[1,1];

#         accuracy1=(tp+tn)/total1

        _val_specificity = tn/(fp+tn)
        _val_sensitivity = tp/(tp+fn)
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print(f'\n — val_f1: {_val_f1} — val_precision: {_val_precision} — val_recall {_val_recall}')
        print(f' — val_sensitivity: {_val_sensitivity} — _val_specificity: {_val_specificity}\n')
        
        return



dfROC =  pd.DataFrame(data=None, columns=['AREA_ROC'] )   
dfROC_best =  pd.DataFrame(data=None, columns=['AREA_ROC'] )   
dfROC_train =  pd.DataFrame(data=None, columns=['AREA_ROC'] )   
dfROC_best_train =  pd.DataFrame(data=None, columns=['AREA_ROC'] )   

dfPR  =  pd.DataFrame(data=None, columns=['AREA_PR' ] )
dfPR_best  =  pd.DataFrame(data=None, columns=['AREA_PR' ] )
dfPR_train  =  pd.DataFrame(data=None, columns=['AREA_PR' ] )
dfPR_best_train  =  pd.DataFrame(data=None, columns=['AREA_PR' ] )


dfCM      = pd.DataFrame(data=None, columns=['tp', 'fn' , 'fp' , 'tn'])
dfCM_best = pd.DataFrame(data=None, columns=['tp', 'fn' , 'fp' , 'tn'])
dfCM_train      = pd.DataFrame(data=None, columns=['tp', 'fn' , 'fp' , 'tn'])
dfCM_best_train = pd.DataFrame(data=None, columns=['tp', 'fn' , 'fp' , 'tn'])    

training_histories = []
last_model_performance_histories = []
best_model_performance_histories = []

last_model_performance_histories_trainset = []
best_model_performance_histories_trainset = []


skf = StratifiedKFold( n_splits= k_folds , random_state = 23, shuffle=True)
print(skf)  
k=0


for train_index, test_index in skf.split(sequences, labels):
    
    # fold K
    k=k+1;
    
    print()    
    print()    
    print(" --------------- Fold:", k, " ---------------")
    print("train fold size:", len(train_index ), "\nValidation fold size:", len(test_index) )

    # get the features and labels for the train set
    train_features = input_features[train_index]
    train_labels   = labels[train_index]#input_labels[train_index]
    
    # get the features and labels for the validation set
    test_features = input_features[test_index]
    test_labels   = labels[test_index]#input_labels[test_index] 

    # feature dimension
    inputSeqLength = len(train_features[0]);
    inputSeqLength = np.array(train_features).shape[1];
    
    size0 = np.array(train_features).shape[0];
    size1 = np.array(train_features).shape[1];
    print("number of samples in Training: " + str(size0) + "  seq length: " + str(size1))

    size0 = np.array(test_features).shape[0];
    size1 = np.array(test_features).shape[1];
    print("number of samples in Testing : " + str(size0) + "  seq length: " + str(size1))

    size0 = np.array(input_features).shape[0];
    size1 = np.array(input_features).shape[1];
    print("number of samples in used dataset: " + str(size0) + "  seq length: " + str(size1))
        
    print('Building model')

    model, config = make_model()

    learning_rate = config['learning_rate']

#     decay_rate = learning_rate / hyper_epoch

    adamOpt = Adam(lr = 0.0005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad = False)
    model.compile(loss='binary_crossentropy', optimizer=adamOpt, metrics=[sensitivity, specificity,'binary_accuracy']) # 'adam' , metrics=['binary_accuracy', my_pred]
    #model.compile(loss='binary_crossentropy', optimizer=adamOpt, metrics=[sensitivity, specificity,'binary_accuracy']) # 'adam' , metrics=['binary_accuracy', my_pred]
    
    # Adadelta didn't give us good result. we omitted it.
    #model.compile(loss='binary_crossentropy', optimizer='Adadelta', metrics=['binary_accuracy']) # 'adam' , metrics=['binary_accuracy', my_pred]
    
    print ( model.summary() )
    
    
    print('Running at most', hyper_epoch, 'epochs')

    print()
    print("************************************************************ ")        
    print("************** TRAINING START (fold", k, ") **************** ")
    # https://machinelearningmastery.com/check-point-deep-learning-models-keras/
    
    fileWeight_BestModel = f"{snapshot_dir}{k}_Weights_bestModel.h5"
    every_epoch_checkpoint_fname = snapshot_dir+"saved-model-{epoch:02d}-{val_loss:.2f}-fold_"+str(k)+".hdf5"
    
    #     monitor='val_loss', mode='min'
    #     monitor='val_acc', mode='max'
    #     monitor='val_fmeasure', mode='max'
    
    
    best_checkpoint = ModelCheckpoint(fileWeight_BestModel, monitor='val_loss',mode='auto', save_best_only=True, verbose=1, save_weights_only=True )
    every_epoch_checkpoint = ModelCheckpoint(filepath = every_epoch_checkpoint_fname, monitor='val_loss', verbose=1, save_best_only=False, 
                                             save_weights_only=False, mode='auto', period=1)
    #     val_loss,val_binary_accuracy,loss,binary_accuracy
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')

    schedule = SGDRScheduler(min_lr = config['SGDRScheduler_config']['min_lr'],
                             max_lr = config['SGDRScheduler_config']['max_lr'],
                             steps_per_epoch = config['SGDRScheduler_config']['steps_per_epoch'],
                             lr_decay = config['SGDRScheduler_config']['lr_decay'],
                             cycle_length = config['SGDRScheduler_config']['cycle_length'],
                             mult_factor = config['SGDRScheduler_config']['mult_factor'])
    
    # for plotting losses        
    loss_plotter = PlotLosses()

    metrics = Metrics(val_data  = (test_features, test_labels))

    history = model.fit(train_features, train_labels, epochs= hyper_epoch, batch_size=hyper_batch_size, shuffle=True, verbose=1, 
                        validation_data = (test_features, test_labels), callbacks=[schedule, best_checkpoint, every_epoch_checkpoint, loss_plotter, metrics] )    
    #class_weight=class_weight , 
    
    training_histories.append(history)

    # using validation set fold for validation
    #history = model.fit(train_features, train_labels, epochs= hyper_epoch, batch_size=hyper_batch_size, shuffle=True, verbose=1, 
    #                    validation_data = (test_features, test_labels), class_weight=class_weight , callbacks=[early_stopping, checkpoint] )  
    
    
    # save the last iteration's model and weights
    fileModel = f'{snapshot_dir}{k}_Model.h5'
    model.save(fileModel)
    fileWeight = f'{snapshot_dir}{k}_Weights.h5'
    model.save_weights(fileWeight)

    print("************** TRAINING END (fold", k, ") **************** ")
    print("********************************************************** ")        

    
    print("************************************************************************ ")            
    print("************** TESTING START (LAST MODEL) (fold", k, ") **************** ")
    
    last_model_weight_file = f'{snapshot_dir}{k}_Weights.h5'
    print(last_model_weight_file)
    model.load_weights(last_model_weight_file, by_name=False)

    test_summary = perform_test(model, test_features, test_labels, dfCM, dfROC, dfPR )
    last_model_performance_histories.append(test_summary)    
        
    print("************** TESTING END (LAST MODEL) (fold", k, ") **************** ")
    print("************************************************************************ ")            
    
    
    print("************************************************************************ ")            
    print("************** TESTING START (BEST MODEL) (fold", k, ") **************** ")
    best_model_weight_file = f"{snapshot_dir}{k}_Weights_bestModel.h5"
    print(fileWeight_BestModel)
    model.load_weights(best_model_weight_file, by_name=False)

    test_summary = perform_test(model, test_features, test_labels, dfCM_best, dfROC_best, dfPR_best )
    best_model_performance_histories.append(test_summary)    
    
    print("************** TESTING END  (BEST MODEL) (fold", k, ") **************** ")
    print("************************************************************************ ")   

    
    print("************************************************************************ ")            
    print("************** TESTING START ON TRAINING SET (LAST MODEL) (fold", k, ") **************** ")
    last_model_weight_file = f'{snapshot_dir}{k}_Weights.h5'
    print(last_model_weight_file)
    model.load_weights(last_model_weight_file, by_name=False)

    test_summary = perform_test(model, train_features, train_labels, dfCM_train, dfROC_train, dfPR_train )
    last_model_performance_histories_trainset.append(test_summary) 
    
    print("************** TESTING END ON TRAINING SET (LAST MODEL) (fold", k, ") **************** ")
    print("************************************************************************ ")
    
    print("************************************************************************ ")            
    print("************** TESTING START ON TRAINING SET (BEST MODEL) (fold", k, ") **************** ")
    best_model_weight_file = f"{snapshot_dir}{k}_Weights_bestModel.h5"
    print(fileWeight_BestModel)
    model.load_weights(best_model_weight_file, by_name=False)

    test_summary = perform_test(model, train_features, train_labels, dfCM_best_train, dfROC_best_train, dfPR_best_train )
    best_model_performance_histories_trainset.append(test_summary)
    
    print("************** TESTING END ON TRAINING SET (BEST MODEL) (fold", k, ") **************** ")
    print("************************************************************************ ")    
    
##########################  Split End ###############################

StratifiedKFold(n_splits=10, random_state=23, shuffle=True)


 --------------- Fold: 1  ---------------
train fold size: 33546 
Validation fold size: 3728
number of samples in Training: 33546  seq length: 1000
number of samples in Testing : 3728  seq length: 1000


W0728 09:38:02.756501 140094755354432 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0728 09:38:02.826603 140094755354432 nn_ops.py:4224] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


number of samples in used dataset: 37274  seq length: 1000
Building model


W0728 09:38:02.925521 140094755354432 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1000, 4)]    0                                            
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 1000, 64)     832         input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 1000, 64)     1344        input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_9 (Conv1D)               (None, 1000, 32)     1056        input_2[0][0]                    
____________________________________________________________________________________________

UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv1d_7/conv1d}}]]
	 [[metrics_2/sensitivity/Identity/_431]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv1d_7/conv1d}}]]
0 successful operations.
0 derived errors ignored.

<Figure size 432x288 with 0 Axes>

In [ ]:
def perform_test( mdl, features, labels, cm_df, roc_df, dpr_df ):
    
    predicted_labels = (np.asarray(mdl.predict(np.stack(features)))).round()
    
    
    #cm = confusion_matrix(np.argmax(labels, axis=1), np.argmax(predicted_labels, axis=1))
    cm = confusion_matrix(labels, predicted_labels)
    output = ""
    output = output + 'Confusion matrix:\n' + str(cm) 
    output += "\n"
    
    print('Confusion matrix:\n' ,cm)
    # cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
    total1=sum(sum(cm))
    tp=cm[0,0];fn=cm[0,1];fp=cm[1,0];tn=cm[1,1];
    
    accuracy1=(tp+tn)/total1
    sensitivity1 = tp/(tp+fn)
    specificity1 = tn/(fp+tn)
    if (tp)==0:
        precision1 = 0
    else:
        precision1 = tp / (tp + fp)

    MCC = (tp*tn - fp*fn) / np.sqrt(  (tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)  )    
    print( "Precision : " + str(precision1) + " Sensitivity/Recall: " + str (sensitivity1) + " Specificity: " + str(specificity1 )  + " Accuracy: " + str(accuracy1 ) + " MCC: " + str(MCC )) 

    output = output + "Precision : " + str(precision1) + " Sensitivity/Recall: " + str (sensitivity1) + " Specificity: " + str(specificity1 )  + " Accuracy: " + str(accuracy1 ) + " MCC: " + str(MCC )
    output += "\n"
    
    cm_df =  dfCM.append( {  'tp': tp , 'fn' : fn , 'fp': fp  , 'tn' : tn  } , ignore_index=True)   
    
    #auc_roc           = roc_auc_score( np.argmax(labels, axis=1), np.argmax(predicted_labels, axis=1) )  
    auc_roc           = roc_auc_score( labels, predicted_labels )  
    #average_precision = average_precision_score(np.argmax(labels, axis=1), np.argmax(predicted_labels, axis=1))
    average_precision = average_precision_score(labels, predicted_labels )
    print( "area ROC : " , auc_roc ,  " area PR " , average_precision)
    output = output + " area ROC : " + str(auc_roc) +  " area PR " + str(average_precision)
    output += "\n"

    roc_df =  roc_df.append( {  'AREA_ROC': auc_roc  } , ignore_index=True) 
    dpr_df  =  dpr_df.append( {  'AREA_PR': average_precision  } , ignore_index=True) 
    
    return output

In [ ]:
def test_metrics():
    k = 1
    print("************************************************************************ ")            
    print("************** TESTING START (LAST MODEL) (fold", k, ") **************** ")
    
    last_model_weight_file = f'{snapshot_dir}saved-model-114-0.40-fold_{k}.hdf5'
    print(last_model_weight_file)
    model.load_weights(last_model_weight_file, by_name=False)

    test_summary = perform_test(model, test_features, test_labels, dfCM, dfROC, dfPR )
    last_model_performance_histories.append(test_summary)    
        
    print("************** TESTING END (LAST MODEL) (fold", k, ") **************** ")
    print("************************************************************************ ")            
   
    
    print("************************************************************************ ")            
    print("************** TESTING START (BEST MODEL) (fold", k, ") **************** ")
    best_model_weight_file = f"{snapshot_dir}{k}_Weights_bestModel.h5"
    print(fileWeight_BestModel)
    model.load_weights(best_model_weight_file, by_name=False)

    test_summary = perform_test(model, test_features, test_labels, dfCM_best, dfROC_best, dfPR_best )
    best_model_performance_histories.append(test_summary)    
    
    print("************** TESTING END  (BEST MODEL) (fold", k, ") **************** ")
    print("************************************************************************ ")   

    
#     print("************************************************************************ ")            
#     print("************** TESTING START ON TRAINING SET (LAST MODEL) (fold", k, ") **************** ")
#     last_model_weight_file = f'{snapshot_dir}{k}_Weights.h5'
#     print(last_model_weight_file)
#     model.load_weights(last_model_weight_file, by_name=False)

#     test_summary = perform_test(model, train_features, train_labels, dfCM_train, dfROC_train, dfPR_train )
#     last_model_performance_histories_trainset.append(test_summary) 
    
#     print("************** TESTING END ON TRAINING SET (LAST MODEL) (fold", k, ") **************** ")
#     print("************************************************************************ ")
    
    print("************************************************************************ ")            
    print("************** TESTING START ON TRAINING SET (BEST MODEL) (fold", k, ") **************** ")
    best_model_weight_file = f"{snapshot_dir}{k}_Weights_bestModel.h5"
    print(fileWeight_BestModel)
    model.load_weights(best_model_weight_file, by_name=False)

    test_summary = perform_test(model, train_features, train_labels, dfCM_best_train, dfROC_best_train, dfPR_best_train )
    best_model_performance_histories_trainset.append(test_summary)
    
    print("************** TESTING END ON TRAINING SET (BEST MODEL) (fold", k, ") **************** ")
    print("************************************************************************ ") 

In [ ]:
test_metrics()

In [ ]:
def plot_training_histories():
    
    for history in training_histories:
        # Plot training & validation accuracy values
#         plt.plot(history.history['binary_accuracy'])
#         plt.plot(history.history['val_binary_accuracy'])
#         plt.title('Model accuracy')
#         plt.ylabel('Accuracy')
#         plt.xlabel('Epoch')
#         plt.legend(['Train', 'Test'], loc='upper left')
#         plt.show()

        # Plot training & validation loss values
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Test'], loc='upper left')
        plt.show()    
        
#plot_training_histories()       

In [ ]:
#
# load a model from file and evaluate its performance
#
#

# dfROC_best_val =  pd.DataFrame(data=None, columns=['AREA_ROC'] )   

# dfPR_best_val  =  pd.DataFrame(data=None, columns=['AREA_PR' ] )

# dfCM_best_val = pd.DataFrame(data=None, columns=['tp', 'fn' , 'fp' , 'tn'])    

# for k in range(1,11):
#     best_model_weight_file = f"{ds_dir}{k}_Weights_bestModel.h5"
#     print("************************* ",k, " **************************")
#     print(fileWeight_BestModel)
#     model.load_weights(best_model_weight_file, by_name=False)

#     perform_test(model, test_features, test_labels, dfCM_best_val, dfROC_best_val, dfPR_best_val )
#     print()
#     print()
    

In [ ]:
# generate loss plots for all the folds along with the corresponding performance metrics
# and write the result to a PDF file

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np

with PdfPages('test.pdf') as pdf:
    t = np.arange(0.0, 2.0, 0.01)
    s = 1 + np.sin(2*np.pi*t)
    s = s * 50

    fig = plt.figure(figsize=(12,12))
    n=0
    
    for history in training_histories:
        n += 1
        ax = fig.add_subplot(4,3,n)

        # Plot training & validation loss values
        ax.plot(history.history['loss'])
        ax.plot(history.history['val_loss'])
        ax.yaxis.set_label_text('Loss')
        ax.xaxis.set_label_text('Epoch')
        ax.legend(['Train', 'Test'], loc='upper right')            

    pdf.savefig(fig)

In [ ]:
print('*********** AREA ROC for BEST model *************')

for index, row in dfROC_best.iterrows():
        print(  row['AREA_ROC']  )
        
print( dfROC_best.mean(axis = 0, skipna = True)  )

print('*********** AREA ROC for BEST model END *************')


print('*********** AREA PR for BEST model *************')

for index, row in dfPR_best.iterrows():
        print(  row['AREA_PR']  )
        
print( dfPR_best.mean(axis = 0, skipna = True)  )

print('*********** AREA PR for BEST model END *************')

In [ ]:
print('*********** CV  for last iteration model *************')

for index, row in dfCM.iterrows():
        print(  row['tp'], row['fn'] ,  row['fp'], row['tn']  )

print( dfCM.sum(axis = 0, skipna = True)  )

dfTotal=dfCM.sum(axis = 0, skipna = True) 
total_tp = dfTotal['tp'];
total_fn = dfTotal['fn'];
total_fp = dfTotal['fp'];
total_tn = dfTotal['tn'];
total_all = total_tp + total_fn + total_fp + total_tn;
accuracy1=(total_tp+total_tn)/total_all
sensitivity1 = total_tp/(total_tp+total_fn)
specificity1 = total_tn/(total_fp+total_tn)
if (total_tp)==0:
    precision1 = 0
else:
    precision1 = total_tp / (total_tp + total_fp)

MCC = (total_tp*total_tn - total_fp*total_fn) / np.sqrt(  (total_tp+total_fp)*(total_tp+total_fn)*(total_tn+total_fp)*(total_tn+total_fn)  ) 

print("TOTAL RECORDS IN ALL FOLD OF CV " , total_all)
print(  "Precision : " + str(precision1) + " Sensitivity/Recall: " + str (sensitivity1) + " Specificity: " + str(specificity1 )  + " Accuracy: " + str(accuracy1 ) + " MCC: " + str(MCC ))



print('*********** CV  for best model *************')
for index, row in dfCM_best.iterrows():
        print(  row['tp'], row['fn'] ,  row['fp'], row['tn']  )
        
        
print( dfCM_best.sum(axis = 0, skipna = True)  )

dfTotal=dfCM_best.sum(axis = 0, skipna = True) 
total_tp = dfTotal['tp'];
total_fn = dfTotal['fn'];
total_fp = dfTotal['fp'];
total_tn = dfTotal['tn'];
total_all = total_tp + total_fn + total_fp + total_tn;
accuracy1=(total_tp+total_tn)/total_all
sensitivity1 = total_tp/(total_tp+total_fn)
specificity1 = total_tn/(total_fp+total_tn)
if (total_tp)==0:
    precision1 = 0
else:
    precision1 = total_tp / (total_tp + total_fp)

MCC = (total_tp*total_tn - total_fp*total_fn) / np.sqrt(  (total_tp+total_fp)*(total_tp+total_fn)*(total_tn+total_fp)*(total_tn+total_fn)  ) 

print("TOTAL RECORDS IN ALL FOLD OF CV " , total_all)
print(  "Precision : " + str(precision1) + " Sensitivity/Recall: " + str (sensitivity1) + " Specificity: " + str(specificity1 )  + " Accuracy: " + str(accuracy1 ) + " MCC: " + str(MCC ))


In [ ]:
##########################  PR curve Drawing ###############################

test_label_rep = test_labels
pred_data = predicted_labels
cm1 = confusion_matrix(np.argmax(test_labels, axis=1), np.argmax(predicted_labels, axis=1))


print("********* Precision-Recall Curve *********")

average_precision = average_precision_score(np.argmax(test_labels, axis=1), np.argmax(predicted_labels, axis=1))
print('Average Precision (AUPR)' , average_precision)
precision, recall, _ = precision_recall_curve(np.argmax(test_labels, axis=1), np.argmax(predicted_labels, axis=1))


plt.step(recall, precision, color='b', alpha=0.2, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2, color='b')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])

##########################  PR curve Drawing END ###############################

In [ ]:

##########################  ROC curve Drawing START ###############################
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html



print("********* Receiver Operating Curve *********")
print ('AUC: '+str(roc_auc_score( np.argmax(test_labels, axis=1), np.argmax(predicted_labels, axis=1) ) ) )

fpr, tpr, thresholds = roc_curve(  np.argmax(test_labels, axis=1), np.argmax(predicted_labels, axis=1)  )
roc_auc = auc(fpr, tpr)

lw=2;
plt.plot(fpr, tpr, lw=lw, alpha=0.3, label='ROC'  )
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()


##########################  ROCcurve Drawing END ###############################

In [ ]:
########################## Cross-validated ROC curve Drawing START ###############################
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html#sphx-glr-auto-examples-model-selection-plot-roc-crossval-py


print(cm)
print(history)

import json
# Get the dictionary containing each metric and the loss for each epoch
#history_dict = history
#print(history_dict['loss'][49])

########################## Cross-validated ROC curve Drawing END ###############################